# Rainbow Six Siege Cleaning 

In [1]:
import pandas as pd

In [196]:
folder_path = '/Users/natalielewis/Desktop/Ironhack/Gaming_Project/data/raw/rainbow6/player-lists'
all_files = os.listdir(folder_path)
csv_files = [f for f in all_files if f.endswith('.csv')]
rainbow6_players_file_list = []
for csv in csv_files:
    file_path = os.path.join(folder_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        rainbow6_players_file_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            rainbow6_players_file_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

In [233]:
rainbow6_players_concatenated = pd.concat(rainbow6_players_file_list, ignore_index=True)


In [189]:
pd.set_option("display.max_columns", None)

## DataFrame Cleaning

In [234]:
def clean_column_names(df):
    """
    Clean column names of a DataFrame using regex patterns.
    Apply additional cleaning steps to specific columns.
    
    Args:
    df (pandas.DataFrame): DataFrame with columns to be cleaned.
    
    Returns:
    pandas.DataFrame: DataFrame with cleaned column names and data.
    """
    # Rename specific columns
 #
    
    # Drop specific columns
    df = df.drop(columns=["Unnamed: 0","GENERAL - COMBAT.PVPBlindKills","GENERAL - COMBAT.PVPPenetrationKills","GENERAL - COMBAT.PVPBulletsHit","GENERAL - COMBAT.PVPBulletsFired",
                          "GENERAL - COMBAT.PVPRappelBreach","GENERAL - COMBAT.PVPDBNOs","GENERAL - TEAM PLAY.PVPDBNOAssists","GENERAL - TEAM PLAY.PVPHostagesRescued","GENERAL - TEAM PLAY.PVPHostageDefense"
                          ,"GENERAL - TEAM PLAY.PVPReinforcementsDeployed","GENERAL - TEAM PLAY.PVPBarricadesDeployed","GENERAL - TEAM PLAY.PVPGadgetsDestroyed","GENERAL - TEAM PLAY.PVPServerAggression"
                          ,"GENERAL - TEAM PLAY.PVPServerDefender","GENERAL - TEAM PLAY.PVPServersHacked","GENERAL - TEAM PLAY.PVPReviveDenied","CASUAL.CasualTimePlayed","CASUAL.CasualWins"
                          ,"CASUAL.CasualLosses","CASUAL.CasualMatches","CASUAL.CasualKDRatio","CASUAL.CasualKillsPerMatch","CASUAL.CasualKillsPerMinute","RANKED.RankedTimePlayed","RANKED.RankedWins"
                          ,"RANKED.RankedLosses","RANKED.RankedMatches","RANKED.RankedKDRatio","RANKED.RankedKillsPerMatch","RANKED.RankedKillsPerMinute","GENERAL - COMBAT.PVPAccuracy","GENERAL.PVPMatchesPlayed","GENERAL.PVPTotalXp"
                          ,"CASUAL.CasualDeaths", "CASUAL.CasualKills", "CASUAL.CasualWLRatio","RANKED.RankedDeaths", "RANKED.RankedKills","GENERAL - COMBAT.PVPSuicides","GENERAL - COMBAT.PVPMeleeKills"
                          ,"RANKED.RankedWLRatio"])
    #df = df.drop(columns=['index'])
    
    # Clean column names using regex patterns
    cleaned_columns = df.columns.str.replace(r'_?COMBAT\.|[_ ]TEAM[_ ]PLAY\.|RANKED\.|PVP|GENERAL ?\.?-?|CASUAL\.', '', regex=True)
    cleaned_columns = cleaned_columns.str.lstrip('_')  # Remove leading underscores
    df.columns = cleaned_columns
    
    # Apply additional cleaning steps to specific columns
    df['WLRatio'] = df['WLRatio'].str.replace('%', '').astype(float)
    # Assuming 'column_name' is the name of the column you want to clean
    for column in df.columns:
     # Check if the data type of the column is object (string)
        if df[column].dtype == 'object':
            # If the data type is object, remove commas
            df[column] = df[column].str.replace(',', '')
            # Convert the column to type integer if it contains numeric values
            if df[column].str.isnumeric().all():
                # Fill missing values with 0
                df[column] = df[column].fillna(0)
                # Convert the column to type integer
                df[column] = df[column].astype(int)
    
    return df

In [235]:
# Function to extract hours from time string
def extract_hours(time_str):
    if isinstance(time_str, str):
        parts = time_str.split()
        for part in parts:
            if part.endswith('H'):
                return int(part[:-1])
        return 0  # Return 0 if no 'H' part found
    else:
        return 0

In [236]:
cleaned_rainbow = clean_column_names(rainbow6_players_concatenated)
cleaned_rainbow['HoursPlayed'] = cleaned_rainbow['TimePlayed'].map(extract_hours)
cleaned_rainbow.drop(columns=["TimePlayed"],inplace=True)
duplicates = cleaned_rainbow[cleaned_rainbow.duplicated(subset=['name'], keep=False)]
cleaned_rainbow.drop_duplicates(subset=['name'], keep='first', inplace=True)

## Final Outcome

In [237]:
cleaned_rainbow

,name,KDRatio,Deaths,Headshots,MatchesWon,MatchesLost,WLRatio,Kills,KillAssists,Revives,HoursPlayed
0,zFanfarrao,2.05,66421,35092,28861,4817,85.3,136175,28643,2145,6925
1,x.C.H.R.I.S.x,1.16,143069,31845,28379,23204,51.3,165987,24511,1566,13287
2,ZootvonKlaus,2.00,68313,39249,27832,5008,75.2,136737,25562,3263,8904
3,x ranl x,1.34,102664,42872,26386,9149,72.2,137081,39054,2374,7728
4,B U T C H E R,1.82,60816,26573,25350,4590,83.7,110821,31855,2143,6647
...,...,...,...,...,...,...,...,...,...,...,...
572,QKapwn,0.81,110785,18172,12700,12850,49.1,89781,18048,2069,8069
573,Voyager.,1.25,48431,15961,12679,5197,70.2,60722,12058,1380,4337
574,ES-CON-FIELD,0.81,77338,12252,12673,12645,48.4,62962,21036,2635,6625
575,Philias.,1.38,80287,42009,12659,8454,59.6,111124,24583,1716,6108


#### Dropping Bots

## Importing the top Operators per player

In [238]:
folder_path = '/Users/natalielewis/Desktop/Ironhack/Gaming_Project/data/raw/rainbow6/player_top_operators'
all_files = os.listdir(folder_path)
csv_files = [f for f in all_files if f.endswith('.csv')]
operators_file_list = []
for csv in csv_files:
    file_path = os.path.join(folder_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        operators_file_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            operators_file_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

In [254]:
top_operators_concatenated

,Player Name,Top Operator 1,Top Operator 2,Top Operator 3
0,zFanfarrao,TWITCH,THERMITE,BANDIT
1,x.C.H.R.I.S.x,ASH,LESION,ZOFIA
2,ZootvonKlaus,ROOK,THERMITE,DOKKAEBI
3,x ranl x,ROOK,IQ,FINKA
4,B U T C H E R,BLITZ,THERMITE,FROST
...,...,...,...,...
601,Gamer.0.,FROST,ASH,THERMITE
602,Belenos279,FROST,TWITCH,MUTE
603,VillainInBlue,ROOK,FUZE,ASH
604,Dizel_Max,BANDIT,THERMITE,SLEDGE


In [239]:
top_operators_concatenated = pd.concat(operators_file_list, ignore_index=True)


In [257]:
top_operators_concatenated.columns

Index(['Player Name', 'Top Operator 1', 'Top Operator 2', 'Top Operator 3'], dtype='object')

#### Operator Cleaning

In [ ]:
top_operators_concatenated = top_operators_concatenated.drop(columns=["Unnamed: 0"])


In [262]:
top_operators_concatenated.rename(columns={'Player Name': "name"}, inplace=True)

In [245]:
non_empty_rows_count = top_operators_concatenated.apply(lambda row: not row.eq("").any(), axis=1).sum()
print("Number of non-empty rows:", non_empty_rows_count)

Number of non-empty rows: 606


In [250]:
top_operators_concatenated['Player Name'].duplicated().sum()

56

In [258]:
duplicates = top_operators_concatenated[top_operators_concatenated.duplicated(subset=['Player Name'], keep=False)]

In [259]:
top_operators_concatenated.drop_duplicates(inplace=True)

In [260]:
top_operators_concatenated.dropna(inplace=True)

### Merging Player Stats and Operators

In [263]:
top_operators_concatenated

,name,Top Operator 1,Top Operator 2,Top Operator 3
0,zFanfarrao,TWITCH,THERMITE,BANDIT
1,x.C.H.R.I.S.x,ASH,LESION,ZOFIA
2,ZootvonKlaus,ROOK,THERMITE,DOKKAEBI
3,x ranl x,ROOK,IQ,FINKA
4,B U T C H E R,BLITZ,THERMITE,FROST
...,...,...,...,...
600,FENRRIR-_-,THERMITE,DOC,JÄGER
601,Gamer.0.,FROST,ASH,THERMITE
602,Belenos279,FROST,TWITCH,MUTE
603,VillainInBlue,ROOK,FUZE,ASH


In [264]:
rainbow_df = pd.merge(cleaned_rainbow, top_operators_concatenated, on='name', how='inner')


In [265]:
rainbow_df

,name,KDRatio,Deaths,Headshots,MatchesWon,MatchesLost,WLRatio,Kills,KillAssists,Revives,HoursPlayed,Top Operator 1,Top Operator 2,Top Operator 3
0,zFanfarrao,2.05,66421,35092,28861,4817,85.3,136175,28643,2145,6925,TWITCH,THERMITE,BANDIT
1,x.C.H.R.I.S.x,1.16,143069,31845,28379,23204,51.3,165987,24511,1566,13287,ASH,LESION,ZOFIA
2,ZootvonKlaus,2.00,68313,39249,27832,5008,75.2,136737,25562,3263,8904,ROOK,THERMITE,DOKKAEBI
3,x ranl x,1.34,102664,42872,26386,9149,72.2,137081,39054,2374,7728,ROOK,IQ,FINKA
4,B U T C H E R,1.82,60816,26573,25350,4590,83.7,110821,31855,2143,6647,BLITZ,THERMITE,FROST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,QKapwn,0.81,110785,18172,12700,12850,49.1,89781,18048,2069,8069,VALKYRIE,MUTE,GLAZ
532,Voyager.,1.25,48431,15961,12679,5197,70.2,60722,12058,1380,4337,ASH,JÄGER,DOC
533,ES-CON-FIELD,0.81,77338,12252,12673,12645,48.4,62962,21036,2635,6625,FROST,LESION,THATCHER
534,Philias.,1.38,80287,42009,12659,8454,59.6,111124,24583,1716,6108,BANDIT,BLACKBEARD,SLEDGE


In [266]:
rainbow_df.to_csv('merged_rainbow.csv')

# BF2042 Cleaning

In [128]:
import os

In [129]:
folder_path = '/Users/natalielewis/Desktop/Ironhack/Gaming_Project/data/raw/bf2042'
all_files = os.listdir(folder_path)
csv_files = [f for f in all_files if f.endswith('.csv')]
bf_2042_players_file_list = []
for csv in csv_files:
    file_path = os.path.join(folder_path, csv)
    try:
        # Try reading the file using default UTF-8 encoding
        df = pd.read_csv(file_path)
        bf_2042_players_file_list.append(df)
    except UnicodeDecodeError:
        try:
            # If UTF-8 fails, try reading the file using UTF-16 encoding with tab separator
            df = pd.read_csv(file_path, sep='\t', encoding='utf-16')
            bf_2042_players_file_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")

In [131]:
bf_2042_players_concatenated = pd.concat(bf_2042_players_file_list, ignore_index=True)

In [132]:
bf_2042_players_concatenated

,Unnamed: 0,Dmg/Min,K/D,HS%,Win %,Human K/D,Kills,Assists,Deaths,Kills/Min,...,Road Kills,Melee Kills,Vehicle Kills,Gadget Kills,Scoped Kills,Hipfire Kills,Human Kills,AI Kills,Top 3 Specialists,name
0,0,140,1.60,27.5%,54.9%,1.53,"119,835","54,654","75,055",0.90,...,"2,883","1,273","29,041",0,"64,271","14,120","114,825","5,018","['Angel', 'Boris', 'Crawford']",overpesa
1,1,283,8.92,23.4%,72.2%,8.48,"328,180","184,081","36,787",2.42,...,337,899,"261,657",0,"57,680","4,095","311,892","16,295","['Zain', 'Mackay', 'Lis']",TOMBSTONE_ADEM
2,2,251,2.43,23.6%,61.2%,2.33,"247,514","65,154","101,968",1.94,...,"1,061","1,072","38,869",0,"153,701","36,518","237,209","10,319","['Mackay', 'Angel', 'Falck']",VLKRxAntonAlonso
3,3,13,4.50,65.2%,42.4%,4.45,"13,692","76,646","3,040",0.07,...,7,5,"1,012",0,"8,307","4,112","13,514",178,"['Casper', 'Rao', 'Boris']",Vi-ka75
4,4,90,1.44,23.5%,53.7%,1.32,"103,796","85,528","72,210",0.64,...,838,"1,389","10,728",0,"59,514","21,306","95,238","8,606","['Mackay', 'Boris', 'Crawford']",Petrosco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,30,216,3.48,19.7%,63.3%,3.13,"244,496","79,741","70,202",1.83,...,98,77,"30,150",0,"182,375","19,933","219,515","24,987","['Mackay', 'Angel', 'Falck']",MMsuperman0216
215,31,50,0.82,33.9%,47.3%,0.73,"58,259","157,859","71,215",0.26,...,"2,263",557,"16,839",0,"23,911","1,626","51,785","6,500","['Boris', 'Lis', 'Blasco']",oleg2oleg
216,32,192,3.89,30.1%,76.6%,3.78,"204,549","106,695","52,516",1.46,...,"1,216",196,"36,313",0,"123,471","27,017","198,381","6,177","['Sundance', 'Mackay', 'Irish']",EVH_Tap_God
217,33,144,1.77,25.4%,61.6%,1.65,"79,295","27,469","44,718",1.08,...,522,665,"8,331",0,"56,736","3,345","73,921","5,406","['Mackay', 'Sundance', 'Angel']",Rudeb0yRonin


In [133]:
bf_2042_players_concatenated.dtypes

Unnamed: 0              int64
Dmg/Min                 int64
K/D                   float64
HS%                    object
Win %                  object
Human K/D             float64
Kills                  object
Assists                object
Deaths                 object
Kills/Min             float64
Kills/Match           float64
Matches Won            object
Matches Lost           object
Damage Dealt           object
Dmg/Match              object
Revives                object
Vehicles Destroyed     object
Multi Kills            object
Headshot Kills         object
Road Kills             object
Melee Kills            object
Vehicle Kills          object
Gadget Kills            int64
Scoped Kills           object
Hipfire Kills          object
Human Kills            object
AI Kills               object
Top 3 Specialists      object
name                   object
dtype: object

In [138]:
def clean_dataframe(df):
    """
    Clean column names of a DataFrame using regex patterns.
    Apply additional cleaning steps to specific columns.
    
    Args:
    df (pandas.DataFrame): DataFrame with columns to be cleaned.
    
    Returns:
    pandas.DataFrame: DataFrame with cleaned column names and data.
    """
    # Rename specific columns
    df = df.rename(columns={"name": "Gamer_Tag"})
    
    # Drop specific columns
    df = df.drop(columns=["Unnamed: 0",'Vehicles Destroyed','Vehicle Kills','Damage Dealt','Scoped Kills','Hipfire Kills','Human Kills','AI Kills','Dmg/Match','Kills/Min','Road Kills','Human K/D','Dmg/Min','HS%','Gadget Kills','Kills/Match'])
    #df = df.drop(columns=['index'])
    #df = df.drop(columns=['Damage Dealt'])
    #df = df.drop(columns=['Scoped Kills'])
    #df = df.drop(columns=['Hipfire Kills'])
    #df = df.drop(columns=['Human Kills'])
    #df = df.drop(columns=['AI Kills'])
    #df = df.drop(columns=['Dmg/Match'])
    #df = df.drop(columns=['Kills/Min'])
    #df = df.drop(columns=['Road Kills'])
    #df = df.drop(columns=['Human K/D'])
    #df = df.drop(columns=['Dmg/Min'])
    #df = df.drop(columns=['HS%'])
    #df = df.drop(columns=['Gadget Kills'])
    #df = df.drop(columns=['Kills/Match'])
    df['Win %'] =  df['Win %'].str.replace('%','')

    
    # Apply additional cleaning steps to specific columns
    # Assuming 'column_name' is the name of the column you want to clean
    for column in df.columns:
     # Check if the data type of the column is object (string)
        if df[column].dtype == 'object':
            # If the data type is object, remove commas
            df[column] = df[column].str.replace(',', '')
            # Convert the column to type integer if it contains numeric values
            if df[column].str.isnumeric().all():
                # Fill missing values with 0
               df[column] = df[column].fillna(0)
                # Convert the column to type integer
               df[column] = df[column].astype(int)

        # Clean column names 
    df.columns = df.columns.str.replace(' ','_')  # Replace spaces with underscores
    df.rename(columns={'Win_%': 'Win_Ratio'}, inplace=True)  # Rename 'Win %' column to 'Win_Ratio'
    
    return df

In [139]:
cleaned_bf = clean_dataframe(bf_2042_players_concatenated)

In [116]:
cleaned_bf

,K/D,Win_Ratio,Kills,Assists,Deaths,Matches_Won,Matches_Lost,Revives,Vehicles_Destroyed,Multi_Kills,Headshot_Kills,Melee_Kills,Vehicle_Kills,Top_3_Specialists,Gamer_Tag
0,5.73,88.8,782522,198027,136581,11407,1435,9931,5050,148462,156018,734,92339,['Mackay' 'Zain' 'Blasco'],-SAGA-Riy
1,2.63,60.4,319848,143093,121750,10766,7045,22656,13657,24968,78567,1458,56103,['Mackay' 'Lis' 'Irish'],lLLlCITMOTORz
2,7.75,82.4,686569,213268,88556,10403,2199,265658,9676,141165,112026,11808,291942,['Falck' 'Angel' 'Paik'],Tygrvs
3,3.88,72.2,343720,150137,88631,9894,3809,27448,17171,35813,46601,1108,103126,['Crawford' 'Sundance' 'Blasco'],HARM CptConquer
4,7.13,85.6,681066,293974,95467,9737,1642,92218,11876,143024,111395,613,141669,['Angel' 'Falck' 'Mackay'],DsciSiv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,3.04,71.3,114133,55784,37538,4078,1641,13624,3408,6251,20354,278,12007,['Mackay' 'Angel' 'Falck'],Prezzbelikeme
215,1.40,78.7,80816,81374,57836,4078,1101,58013,835,10998,12389,86,13538,['Falck' 'Angel' 'Irish'],-SAGA-Miaomiao
216,2.05,63.0,172999,42841,84189,4134,2424,2460,2466,10619,29613,291,19233,['Lis' 'Mackay' 'Falck'],daroadaszi
217,1.44,56.9,98819,33329,68476,4552,3441,7270,6228,5681,8995,604,23198,['Boris' 'Mackay' 'Sundance'],Koupe2Cheveux


## Specialist One Hot Encoding

In [140]:
characters = ['Mackay', 'Zain', 'Blasco', 'Lis', 'Irish','Paik','Falck', 'Sundance','Crawford','Angel','Boris','Dozer','Casper','Rao']

In [141]:
for character in characters:
    cleaned_bf[character] = cleaned_bf['Top_3_Specialists'].apply(lambda x: character in x)

In [142]:
cleaned_bf.columns

Index(['K/D', 'Win_Ratio', 'Kills', 'Assists', 'Deaths', 'Matches_Won',
       'Matches_Lost', 'Revives', 'Multi_Kills', 'Headshot_Kills',
       'Melee_Kills', 'Top_3_Specialists', 'Gamer_Tag', 'Mackay', 'Zain',
       'Blasco', 'Lis', 'Irish', 'Paik', 'Falck', 'Sundance', 'Crawford',
       'Angel', 'Boris', 'Dozer', 'Casper', 'Rao'],
      dtype='object')

In [143]:
cleaned_bf.drop(columns=['Top_3_Specialists'], inplace=True)

In [144]:
cleaned_bf.columns

Index(['K/D', 'Win_Ratio', 'Kills', 'Assists', 'Deaths', 'Matches_Won',
       'Matches_Lost', 'Revives', 'Multi_Kills', 'Headshot_Kills',
       'Melee_Kills', 'Gamer_Tag', 'Mackay', 'Zain', 'Blasco', 'Lis', 'Irish',
       'Paik', 'Falck', 'Sundance', 'Crawford', 'Angel', 'Boris', 'Dozer',
       'Casper', 'Rao'],
      dtype='object')

In [145]:
cleaned_bf

,K/D,Win_Ratio,Kills,Assists,Deaths,Matches_Won,Matches_Lost,Revives,Multi_Kills,Headshot_Kills,...,Irish,Paik,Falck,Sundance,Crawford,Angel,Boris,Dozer,Casper,Rao
0,1.60,54.9,119835,54654,75055,4565,3753,19476,7685,21561,...,False,False,False,False,True,True,True,False,False,False
1,8.92,72.2,328180,184081,36787,4166,1605,6461,98378,14464,...,False,False,False,False,False,False,False,False,False,False
2,2.43,61.2,247514,65154,101968,4166,2643,17551,24556,44819,...,False,False,True,False,False,True,False,False,False,False
3,4.50,42.4,13692,76646,3040,4222,5735,79,220,8102,...,False,False,False,False,False,False,True,False,True,True
4,1.44,53.7,103796,85528,72210,4250,3669,27703,6811,19004,...,False,False,False,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,3.48,63.3,244496,79741,70202,4239,2462,29377,18658,39756,...,False,False,True,False,False,True,False,False,False,False
215,0.82,47.3,58259,157859,71215,4489,4994,4780,5633,8659,...,False,False,False,False,False,False,True,False,False,False
216,3.89,76.6,204549,106695,52516,4233,1294,8419,23654,45327,...,True,False,False,True,False,False,False,False,False,False
217,1.77,61.6,79295,27469,44718,4258,2651,15251,5743,15258,...,False,False,False,True,False,True,False,False,False,False


In [146]:
cleaned_bf.to_csv('bf_2042_playerstats.csv')